# 報告 2022-09-02

## 元データの内訳

In [1]:
%%sh
cat ~/ABCT/comp-proto/Annotation-complete-IDed/BCCWJ-ABC*.psd | munge-trees -w > /tmp/comp-yori.psd
cat ~/ABCT/comp-proto/Annotation-complete-IDed/bccwj_kurabe_*.psd | munge-trees -w > /tmp/comp-kurabe.psd

In [2]:
# 「より」の文数
COUNT_YORI, *_ = ! cat /tmp/comp-yori.psd | wc -l
COUNT_YORI = int(COUNT_YORI)

# 「より」のうち，green（よいパーズ）の数
COUNT_YORI_GREEN, *_ = ! cat /tmp/comp-yori.psd | sed -n -e '/green/p' | wc -l
COUNT_YORI_GREEN = int(COUNT_YORI_GREEN)

# 「より」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_YORI_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_SIMPLE = int(COUNT_YORI_SIMPLE)

# 「より」のうち，連用節の数
COUNT_YORI_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADVERBIAL = int(COUNT_YORI_ADVERBIAL)

# 「より」のうち，連体節の数
COUNT_YORI_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADNOMINAL = int(COUNT_YORI_ADNOMINAL)


In [3]:
# 「比べて」の文数
COUNT_KURABE, *_ = ! cat /tmp/comp-kurabe.psd | wc -l
COUNT_KURABE = int(COUNT_KURABE)

# 「比べて」のうち，green（よいパーズ）の数
COUNT_KURABE_GREEN, *_ = ! cat /tmp/comp-kurabe.psd | sed -n -e '/green/p' | wc -l
COUNT_KURABE_GREEN = int(COUNT_KURABE)

# 「比べて」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_KURABE_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_SIMPLE = int(COUNT_KURABE_SIMPLE)

# 「比べて」のうち，連用節の数
COUNT_KURABE_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADVERBIAL = int(COUNT_KURABE_ADVERBIAL)

# 「比べて」のうち，連体節の数
COUNT_KURABE_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADNOMINAL = int(COUNT_KURABE_ADNOMINAL)

In [4]:
# 集計
import pandas as pd

STAT = pd.DataFrame(
    {
        "全文数": [COUNT_YORI, COUNT_KURABE],
        "GREEN文数": [COUNT_YORI_GREEN, COUNT_KURABE_GREEN],
        "単文数": [COUNT_YORI_SIMPLE, COUNT_KURABE_SIMPLE],
        "連用節数": [COUNT_YORI_ADVERBIAL, COUNT_KURABE_ADVERBIAL],
        "連体節数": [COUNT_YORI_ADNOMINAL, COUNT_KURABE_ADNOMINAL],
    },
    index = ["より", "比べて"]
)

In [5]:
STAT

,全文数,GREEN文数,単文数,連用節数,連体節数
より,2700,693,1023,292,449
比べて,1042,1042,509,123,87


In [6]:
# 合計
STAT.sum()

全文数        3742
GREEN文数    1735
単文数        1532
連用節数        415
連体節数        536
dtype: int64

## 学習データ／評価データ
9:1になるように，事前に分割した。

In [7]:
import datasets

# NOTE: private repoなので，事前にログインが必要。
ds = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
)

ds

/home/owner/.cache/pypoetry/virtualenvs/comparative-ner-utils-sv0RmVnD-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-fe0cb4ac0d530735/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 278.61it/s]


DatasetDict({
    train: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 3368
    })
    test: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 374
    })
})

* 学習データ数： 3,368文
* テストデータ数： 374文

### 学習データの例

#### 単文
```
5_BCCWJ-ABC-aa-simple
妻 が 仕事 に 精出す 一方 、 [[赤沼 は]cont [それ より]prej [もっと]diff [忙しい]deg 。]root

5_BCCWJ-ABC-aa-simple_predicted
[CLS] 妻 が 仕事 に 精 ##出す 一方 、 [赤 ##沼 は]cont [それ より]prej [もっと]diff [忙 ##しい]deg 。 [SEP]


95_bccwj_kurabe_text-aj-simplified
[[旧法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食生活 は [格段 に]diff [豊か]deg に なっ た 。]root

95_bccwj_kurabe_text-aj-simplified_predicted
[CLS] [旧 ##法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食 ##生活 は [格段 に]diff [豊か]deg に なっ た 。 [SEP]

35_bccwj_kurabe_text-ah-simplified
[[アカ ナマコ の 成長 は]cont [アオナマコ に]prej [比べ]prej [若干]diff [劣っ]deg て い た 。]root

35_bccwj_kurabe_text-ah-simplified_predicted
[CLS] [アカ ナ ##マコ の]cont 成長 は [アオ ##ナ ##マコ に 比べ]prej [若干]diff [劣っ]deg て い た 。 [SEP]
```

#### 連用節
```
23_bccwj_kurabe_text-af-simplified
ところが 、 一 九 八 五 年 九月 の プラザ 合意 以降 、 [[円 が]cont [ドル に 比べ て]prej [百 ％ 以上]diff [はね上がり]deg]root 、 突然 日本 は アメリカ より はるか に 高 コスト の 国 に なり まし た 。

23_bccwj_kurabe_text-af-simplified_predicted
[CLS] ところが 、 一 九 八 五 年 九 ##月 の プラザ 合意 以降 、 [円 が]cont [ドル に 比べ て]prej [百 % 以上]diff [はね ##上がり]deg 、 突然 [日本 は]cont [アメリカ より]prej [はるか に]diff [高 コスト]deg の 国 に なり まし た 。 [SEP]
```

#### 連体節
```
21_BCCWJ-ABC-as-simple
三 十 歳 の サラリーマン が [自分 より]prej [七]diff [、]diff [[[八 歳]diff [年下]deg]cont]root の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。

21_BCCWJ-ABC-as-simple_predicted
[CLS] 三 十 歳 の サラリーマン [が]cont [自分 より]prej [七 、 八 歳]diff [年下]deg の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。 [SEP]
```
（\[が\]contが変）

```
99_BCCWJ-ABC-au-simple
何 畳 ある か わから ない くらい 、 [[[教室 より も]prej [広い]deg]cont]root 部屋 。

99_BCCWJ-ABC-au-simple_predicted
[CLS] 何 畳 ある か わから ない くらい 、 [教室 より も]prej [広い]deg 部屋 。 [SEP]

## モデルの設定
* 使用した事前学習モデル： https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking
* このモデルの上で，NER（固有表現認識）の一種として，与えられた文のどのspanが，比較構文の要素に相当するのかについてのモデルを構築。
* 比較構文の要素：
    * prej(acent)：「より」句
    * cont(rast)：比較対象
    * diff(erence)：差の表現
    * deg(ree)：程度表現
    * root：比較構文の最大スコープ（NERモデルにおいては取り除いた）
* 例： [ [太郎が]cont [花子よりも]prej [3cm]diff [高い]deg ]root ことは意外だった。

In [8]:
# 学習パラメータ

training_args = dict(
    # output_dir = str(output_path),

    # エポック数
    num_train_epochs = 27,

    # バッチのサイズ
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 128,

    # 学習率
    learning_rate = 5e-5,
    
    warmup_steps = 200,
    weight_decay = 0,
    # save_strategy = IntervalStrategy.STEPS,
    save_steps = 1000,
    do_eval = True,
    # evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 109,
    include_inputs_for_metrics = True,

    # 乱数シード
    seed = 2630987289,

    # logging_dir = str(output_path / "logs"),
    logging_steps= 10,
)

## 学習結果

In [9]:
result = {
    "score_spanwise_details": {
        "prej": {
            "CORRECT": 166,
            "WRONG_SPAN": 22,
            "SPURIOUS": 19,
            "MISSING": 5,
            "WRONG_LABEL_SPAN": 1
        },
        "diff": {
            "CORRECT": 41,
            "MISSING": 9,
            "SPURIOUS": 10,
            "WRONG_SPAN": 6
        },
        "deg": {
            "CORRECT": 151,
            "SPURIOUS": 36,
            "MISSING": 22,
            "WRONG_LABEL_SPAN": 2,
            "WRONG_SPAN": 15,
            "WRONG_LABEL": 3
        },
        "cont": {
            "CORRECT": 55,
            "SPURIOUS": 50,
            "MISSING": 29,
            "WRONG_SPAN": 28,
            "WRONG_LABEL": 3,
            "WRONG_LABEL_SPAN": 2
        }
    },
    "score_spanwise": {
        "prej": {
            "possible_entries": 194,
            "actual_entries": 208,
            "precision_strict": 0.7980769230769231,
            "recall_strict": 0.8556701030927835,
            "F1_strict": 0.880597014925373,
            "precision_partial": 0.8509615384615384,
            "recall_partial": 0.9123711340206185
        },
        "diff": {
            "possible_entries": 56,
            "actual_entries": 57,
            "precision_strict": 0.7192982456140351,
            "recall_strict": 0.7321428571428571,
            "F1_strict": 0.7787610619469025,
            "precision_partial": 0.7719298245614035,
            "recall_partial": 0.7857142857142857
        },
        "deg": {
            "possible_entries": 193,
            "actual_entries": 207,
            "precision_strict": 0.7294685990338164,
            "recall_strict": 0.7823834196891192,
            "F1_strict": 0.7925,
            "precision_partial": 0.7657004830917874,
            "recall_partial": 0.8212435233160622
        },
        "cont": {
            "possible_entries": 117,
            "actual_entries": 138,
            "precision_strict": 0.39855072463768115,
            "recall_strict": 0.4700854700854701,
            "F1_strict": 0.5411764705882353,
            "precision_partial": 0.5,
            "recall_partial": 0.5897435897435898
        }
    },
    "score_spanwise_F1_strict": 0.7482586368651276,
    "score_tokenwise": {
        "IGNORE": {
            "precision": 0.0,
            "recall": 0.0,
            "f1-score": 0.0,
            "support": 0
        },
        "O": {
            "precision": 0.9357241379310345,
            "recall": 0.9011689691817216,
            "f1-score": 0.9181215320070375,
            "support": 3764
        },
        "B-deg": {
            "precision": 0.7830188679245284,
            "recall": 0.8469387755102041,
            "f1-score": 0.8137254901960785,
            "support": 196
        },
        "B-prej": {
            "precision": 0.8663366336633663,
            "recall": 0.8536585365853658,
            "f1-score": 0.85995085995086,
            "support": 205
        },
        "B-cont": {
            "precision": 0.6198347107438017,
            "recall": 0.7142857142857143,
            "f1-score": 0.6637168141592921,
            "support": 105
        },
        "B-diff": {
            "precision": 0.7931034482758621,
            "recall": 0.7666666666666667,
            "f1-score": 0.7796610169491527,
            "support": 60
        },
        "I-deg": {
            "precision": 0.6610169491525424,
            "recall": 0.75,
            "f1-score": 0.7027027027027027,
            "support": 52
        },
        "I-prej": {
            "precision": 0.8687572590011614,
            "recall": 0.9385194479297365,
            "f1-score": 0.902291917973462,
            "support": 797
        },
        "I-cont": {
            "precision": 0.6470588235294118,
            "recall": 0.7009803921568627,
            "f1-score": 0.6729411764705882,
            "support": 408
        },
        "I-diff": {
            "precision": 0.7272727272727273,
            "recall": 0.8333333333333334,
            "f1-score": 0.7766990291262137,
            "support": 48
        },
        "micro avg": {
            "precision": 0.881455190771961,
            "recall": 0.881455190771961,
            "f1-score": 0.8814551907719611,
            "support": 5635
        },
        "macro avg": {
            "precision": 0.6902123557494436,
            "recall": 0.7305551835649605,
            "f1-score": 0.7089810539535387,
            "support": 5635
        },
        "weighted avg": {
            "precision": 0.885800660585663,
            "recall": 0.881455190771961,
            "f1-score": 0.8829757316610416,
            "support": 5635
        }
    }
}

In [10]:
# データカウント
df_res_count = pd.DataFrame.from_dict(
    result["score_spanwise_details"],
    orient = "index",
)

df_res_count

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL_SPAN,WRONG_LABEL
prej,166,22,19,5,1.0,NaN
diff,41,6,10,9,NaN,NaN
deg,151,15,36,22,2.0,3.0
cont,55,28,50,29,2.0,3.0


* CORRECT: ぴったり
* WRONG_SPAN: spanにずれがある。
* SPURIOUS: 正解データにないspanを予測してしまっている。
* MISSING: 正解データにあるspanを予測できていない。
* WRONG_LABEL_SPAN: spanにずれがあり，かつ，ラベルも間違っている。
* WRONG_LABEL: spanにずれはないが，ラベルが間違っている。

In [11]:
# 統計
df_res_stat = pd.DataFrame.from_dict(
    result["score_spanwise"],
    orient = "index",
)

df_res_stat

,possible_entries,actual_entries,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial
prej,194,208,0.798077,0.855670,0.880597,0.850962,0.912371
diff,56,57,0.719298,0.732143,0.778761,0.771930,0.785714
deg,193,207,0.729469,0.782383,0.792500,0.765700,0.821244
cont,117,138,0.398551,0.470085,0.541176,0.500000,0.589744


### 凡例

* `possible_entries`: 予測されたspanの数
    ```
    ct["CORRECT"] + ct["WRONG_SPAN"] + ct["WRONG_LABEL"] + ct["WRONG_LABEL_SPAN"] + ct["MISSING"]
    ```
* `actual_entires`: テストデータにあるspanの数
    ```
    res["possible_entries"] - ct["MISSING"] + ct["SPURIOUS"]
    ```
* `precision_strict`: 予測のうち，当たっているものの数
    ```
    ct["CORRECT"] / res["actual_entries"]
    ```
* `recall_strict`: テストデータにあるもののうち，予測されたspanの数
    ```
    ct["CORRECT"] / res["possible_entries"]
    ```
* `precision_partial`：strictよりも緩い。 WRONG_SPANを50%カウントに入れている。
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["actual_entries"]
    ```
* `recall_partial`
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["possible_entries"]
    ```
* F1はprecisionとrecallの調和平均

In [12]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
result["score_spanwise_F1_strict"]

0.7482586368651276